## [1] train / test set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split(X, y, test_size, train_size, random_state, shuffle, stratify)
    # arrays : 분할 대상 데이터
    # test_size / train_size : 분할 비율
    # shuffle : 순차적 문할여부(무작위 = True, 순차적 = False)
    # stratify : 모수 데이터 항목 비율을 샘플데이터에서도 유지, list 값입력

In [ ]:
# 방안1)
X = df.drop(columns=[target])
y = df[[key, target]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size, train_size, random_state, shuffle, stratify)


# 방안2)
X_train, X_test = train_test_split(df, test_size, train_size, random_state, shuffle, stratify)

X_train = X_train.drop(columns=[target])
y_train = X_train[[key, target]]
X_test = X_test.drop(columns=[target])
y_test = X_test[[key, target]]

## [2] 명목형 변수 전처리 : Encoding

### 1) one_hot encoding : 0 or 1

In [ ]:
# 범주형 데이터 -> 가변수화
pd.get_dummies(df, columns = [], dummy_na = True, sparse=False, drop_first=False, dtype=None)
    # columns = [] : 특정 컬럼만 인코딩 할 경우
    # dummy_na : 결측값 인코딩 여부 (default = False, True : 결측값 표시 컬럼도 생성됨)
    # 데이터 항목 수 + 1(dymmy_na = True) 만큼 컬럼이 생성되며, 각 컬럼은 0 또는 1의 값을 가짐

In [41]:
## 예시
df = pd.DataFrame({'과일' : ["사과", "배", "딸기", ""]})
pd.get_dummies(df, dummy_na = True)

,과일_,과일_딸기,과일_배,과일_사과,과일_nan
0,0,0,0,1,0
1,0,0,1,0,0
2,0,1,0,0,0
3,1,0,0,0,0


### 2) Label encoding : 0~n

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
## 방안1) fit 한 모델을 새로 정의 하여 원하는 데이터에 transform 진행, 동일한 모델로 피팅가능
# fitting model
le = LabelEncoder()
le = le.fit(train['col'])   #train['col']을 fit

# encoding
train['col'] = le.transform(train['col'])   #train['col']에 따라 encoding
test['col'] = le.transform(test['col'])   #train['col']에 따라 encoding


## 방안2) fit과 transform을 동시에 진행, 동일한 모델로 피팅 할 수 없음
le = LabelEncoder()
train['col'] = le.fit_transform(train['col'])
test['col'] = le.fit_transform(test['col'])

### 3) cat.codes : 0~n

In [ ]:
# .astype('category').cat.codes : 한컬럼씩만 변경가능
df['A'].astype('category').cat.codes
# 특정 컬럼을 수치형으로 변환, 0~(데이터 항목 개수) 만큼 넘버링됨

In [43]:
## 예시
df = pd.DataFrame({'과일' : ["사과", "배", "딸기", ""]})
df['과일'] = df['과일'].astype('category').cat.codes

## [4] 수지형 변수 전처리 : Scaler

### 1) StandardScaler : 표준화 (평균 = 0, 표준편차 = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# fit model
scaler = StandardScaler()
scaler = scaler.fit(X_train)

# transform
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### 2) MinMaxScaler : 정규화 (최대값 = 1, 최소값 = 0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# fit model
scaler = MinMaxScaler()
scaler = scaler.fit(X_train)

# transform
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### 3) 기타

In [ ]:
from sklearn.preprocessing import RobusterScaler
    # 중앙값 = 0 / IQR(1분위(25%) ~ 3분위(75%)) = 1
    # 이상치 영향 최소화

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
    # 0을 기준으로 절대값이 가장 큰 수가 1또는 -1이 되도록 변환
    # 이상치 영향 최소화

## [4] 수치형 변수 전처리 : SimpleImputer(결측치 처리)

In [58]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(missing_values = nan, strategy = 'mean', fill_value=None, verbose=0, copy=True, add_indicator=False)
    # 특정 컬럼을 수치형으로 변환, 0~(데이터 항목 개수) 만큼 넘버링됨missing_values : 빈 값이라고 인식할 값(default = nan)
    # strategy : 'mean', 'median', most_frequent(최빈값), constant(사용자지정, fill_value 필수지정)
    # fill_value : strategy가 constant 인 경우 대체할 값 정의
    # copy : 디폴트 True, True인 경우 복사본을 만들어서 채워주고 False로 지정하면 원 DataFrame에 채움
    # add_indicator : 디폴트 False, 원래 비어있던 값인지 아닌지를 구분하는 구분자 컬럼 추가 여부

In [ ]:
# fit model
imputer = SimpleImputer()
imputer = imputer.fit(train[col])

# transform
train[col] = imputer.transform(train[col])
test[col] = imputer.transform(test[col])

## [5] Model 선정

### 1) 회귀모델 : 종속변수가 연속형

In [67]:
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

# 1) 모든 회귀 모델 함수 정의
LR = LogisticRegression(random_state=42)
RF = RandomForestRegressor(random_state=42)
GVR = GradientBoostingRegressor(random_state=42)
DT = DecisionTreeRegressor(random_state=42)
XGBR = XGBRegressor(random_state=42)
SVR = SVR()


# 2) 모델 피팅
model = model.fit(X_train,y_train)
predict = model.predict(X_test)

In [ ]:
# 3) 모델 성능평가 : MSE(평균제곱오차), RMSE, MAE, MAPE
## MAE(낮을 수록 정확)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,predict)

## MSE / RMSE (낮을 수록 정확) 
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, predict)
RMSE = np.sqrt(mean_squared_error(y_test, predict)) 

## 데이터셋 평가
model.score(X_train, y_train)
model.score(X_test, y_test)

### 2) 분류모델 : 종속변수가 범주형

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRFClassifier


# 1) 모든 분류 모델 함수 정의
LR = LogisticRegression(random_state=42)
RF = RandomForestClassifier(random_state=42)`
GB = GradientBoostingClassifier(random_state=42)
KNN = KNeighborsClassifier()
DT = DecisionTreeClassifier(random_state=42)
XGB = XGBClassifier(random_state=42)
XGBR = XGBRFClassifier(random_state=42)
SVC = SVC()


# 2) 모델 피팅
model = model.fit(X_train,y_train)
predict = model.predict(X_test)

In [ ]:
# 3) 모델 성능평가 : confusion matrix, 정확도, 정밀도, 재현율, F1-score, ROC-AUC
## Confusion matrix 
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_true, predict)
acc = accuracy_score(y_true, predict)  # 정확도
pre = precision_score(y_true, predict)  # 정밀도
recall = recall_score(y_true, predict)  # 재현율

# F1_score
F1_score = f1_score(y_true, y_pred)
F1_score = 2*((pre *reall)/(pre + reall))


## ROC-AUC
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, predict)

--------------------------------------------------------------------

In [ ]:
RandomForestRegressor(n_estimators=10, bootstrap=True, criterion='mse', max_depth=None, min_samples_split=2)
    #  n_estimators :  생성할 트리 개수
    #  bootstrap :  True 베깅, False 페이스팅
    #  criterion : 분할 평가 척도

# fit model
model = model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
model = DecisionTreeRegressor(min_samples_split = 2)
    # n_estimators : 생성할 트리 개수
    # criterion = 부할 성능 측정 지표 ('gini, 'entropy')
    # splitter = 노드에서 분할을 선택하는 방법
    # min_samples_split : 노드 분할을 위한 최소 샘플 데이터 수(클수록 과적합 제어)
    # max_features : 최적 분할을 위해 고려할 feature 개수
    # max_depth : 트리 최대깊이(클수록 과적합 위험)

# fit model
model = model.fit(X_train, y_train)
model.predict(X_test)

# 평가
R-sqaured = model.score(xtrain, ytrain)

# 방안2) 파라미터 최적화를 적용한 모델링
params = {
    'n_estimatiors' : (100,200),
    'max_depth' : (5, 8),
    'min_samples_leaf' : (8, 18),
    'min_samp;es_split' : (8. 16)
}
grid_cv = GridSearchCV(moel, params, cv = 2,  n_jobs=-1)
model_grid_cv = grid_cv.fit(X_train, y_train)
model_grid_cv.predict(X_test)

# 평가
model_grid_cv.best_params_  # 최적 파라미터 조합
model_grid_cv.best_score_   # 최적 예측 정확도
model_grid_cv.feature_importances_  # 변수중요도